In [1]:
import numpy as np
from environment import Environment
from agent import Agent

In [ ]:
EPISODES = 1000 # overnight
MAX_ITER = 10000 # episodes will likely finishing after fewer iterations
EPSILON_SCHEDULE = np.linspace(1, 0.05, num=EPISODES) # descreasing epsilon 
SAVE_EVERY = 1 # save video every # of episodes
LR = 1e-6

drone = Agent(init_position=[10, 10], lr=LR, fov_shape=(9,9), fov_extinguish=(6,6))
env = Environment(random_seed=None)
for episode, eps in zip(range(EPISODES), EPSILON_SCHEDULE):
    
    print(f"{episode}/{EPISODES}")
    env.reset()
    drone.position = np.random.randint(10,90,2) # respawn drone at a random positon
    frame_buffer = []
    
    for it in range(MAX_ITER):
        position = drone.position # drone's (x,y) pos in the env
        action = drone.action(epsilon=eps) # get action

        # environment step (little different than gym's API)
        env.step(position, action)
        drone.observation = env.observe(position, fov=drone._fov_shape) # make observation of env
        reward = env.extinguish(position, fov=drone._fov_extinguish) # reward, normalized in range (-1, 1)
        drone.backprop(action, reward) # backpropagate through policy network

        # render environment using snapshot() method and add to frame buffer
        frame_buffer.append(env.snapshot(drone.position))
         
        # in a terminal state -> next episode.  
        # (This means all fire extinguished, or died out)
        if env.done: break
            
    if episode % SAVE_EVERY == 0: drone.save_video(f"episode-{episode}.mp4", frame_buffer)

0/1000
1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
